# In this notebook, we will be attempting to use transfer learning to predict our labels.

In [78]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16

from pathlib import Path
import imghdr

## Pre-process data

In [79]:
#  the data
data_dir = "./BBTrD/000000001111"
image_extensions = [".png", ".jpg", ".jpeg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

## Load Images and Labels

In [80]:
def load_labels():
    txt_file_path = "labels.txt"
    labels = []
    with open(txt_file_path, 'r') as f:
        for line in f:
            labels.append(line.strip())
    return labels

In [81]:
game_states = load_labels()
game_states = sorted(game_states)
game_states

['000000001111',
 '000000011110',
 '000000101101',
 '000000111100',
 '000001001011',
 '000001011010',
 '000001101001',
 '000001111000',
 '000010000111',
 '000010010110',
 '000010100101',
 '000010110100',
 '000011000011',
 '000011010010',
 '000011100001',
 '000011110000',
 '000100001110',
 '000100101100',
 '000101001010',
 '000101101000',
 '000110000110',
 '000110100100',
 '000111000100',
 '000111100000',
 '001000001101',
 '001000011100',
 '001001001001',
 '001001011000',
 '001010000101',
 '001010010100',
 '001011000001',
 '001011010000',
 '001100001100',
 '001101001000',
 '001110000100',
 '001111000000',
 '010000001011',
 '010000011010',
 '010000101001',
 '010000111000',
 '010010000011',
 '010010010010',
 '010010100001',
 '010010110000',
 '010100001010',
 '010100101000',
 '010110000010',
 '010110100000',
 '011000001001',
 '011000011000',
 '011010000001',
 '011010010000',
 '011100001000',
 '011110000000',
 '100000000111',
 '100000010110',
 '100000100101',
 '100000110100',
 '100001000011

In [82]:
image_height = 128
image_width = 128
batch_size = 81

def load_data():
    # Load Train Data
    train_images = tf.keras.utils.image_dataset_from_directory("./BBTrD/", batch_size=batch_size, image_size=(image_height, image_width), shuffle=False)

    # Load Test Data
    test_images =tf.keras.utils.image_dataset_from_directory("./BBTeD/", batch_size=batch_size, image_size=(image_height, image_width), shuffle=False)
    return train_images, test_images

In [83]:
train_images, test_images = load_data()

Found 81 files belonging to 81 classes.
Found 34 files belonging to 34 classes.


In [84]:
train_images.class_names

['000000001111',
 '000000011110',
 '000000101101',
 '000000111100',
 '000001001011',
 '000001011010',
 '000001101001',
 '000001111000',
 '000010000111',
 '000010010110',
 '000010100101',
 '000010110100',
 '000011000011',
 '000011010010',
 '000011100001',
 '000011110000',
 '000100001110',
 '000100101100',
 '000101001010',
 '000101101000',
 '000110000110',
 '000110100100',
 '000111000100',
 '000111100000',
 '001000001101',
 '001000011100',
 '001001001001',
 '001001011000',
 '001010000101',
 '001010010100',
 '001011000001',
 '001011010000',
 '001100001100',
 '001101001000',
 '001110000100',
 '001111000000',
 '010000001011',
 '010000011010',
 '010000101001',
 '010000111000',
 '010010000011',
 '010010010010',
 '010010100001',
 '010010110000',
 '010100001010',
 '010100101000',
 '010110000010',
 '010110100000',
 '011000001001',
 '011000011000',
 '011010000001',
 '011010010000',
 '011100001000',
 '011110000000',
 '100000000111',
 '100000010110',
 '100000100101',
 '100000110100',
 '100001000011

## Load Pre-trained VGG model

In [85]:
#load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [86]:
num_classes = 81
num_epochs = 10
# we will be freezing the base model's layers in order to keep the weights in the optimal condition they were trained in
for layer in vgg_conv.layers:
    layer.trainable = False

# new classifier model, which consists of the old model's layers plus some new ones
extra_layers = vgg_conv.output
# extra_layers = keras.layers.Conv2D(32, (3, 3), activation='relu')(extra_layers)
extra_layers = keras.layers.Flatten()(extra_layers)
extra_layers = keras.layers.Dense(1024, activation='relu')(extra_layers)

predictions = keras.layers.Dense(num_classes, activation='softmax')(extra_layers)

# create the new model
new_model = keras.models.Model(inputs=vgg_conv.input, outputs=predictions)

# compile the model
new_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
new_model.fit(train_images, epochs=num_epochs)

# print(train_images)





Epoch 1/10
1/1 [==============================] - 20s 20s/step - loss: 48.9703 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 20s 20s/step - loss: 65.0389 - accuracy: 0.0617
Epoch 3/10
1/1 [==============================] - 19s 19s/step - loss: 61.6813 - accuracy: 0.1605
Epoch 4/10
1/1 [==============================] - 19s 19s/step - loss: 63.3740 - accuracy: 0.2099
Epoch 5/10
1/1 [==============================] - 19s 19s/step - loss: 63.8710 - accuracy: 0.2222
Epoch 6/10
1/1 [==============================] - 19s 19s/step - loss: 58.5757 - accuracy: 0.3951
Epoch 7/10
1/1 [==============================] - 18s 18s/step - loss: 56.8728 - accuracy: 0.3580
Epoch 8/10
1/1 [==============================] - 21s 21s/step - loss: 55.7819 - accuracy: 0.4444
Epoch 9/10
1/1 [==============================] - 17s 17s/step - loss: 53.8153 - accuracy: 0.5062
Epoch 10/10
1/1 [==============================] - 18s 18s/step - loss: 51.9743 - accuracy: 0.6049


In [87]:
# testing dropout
# new classifier model, which consists of the old model's layers plus some new ones
extra_layers = vgg_conv.output
# extra_layers = keras.layers.Conv2D(32, (3, 3), activation='relu')(extra_layers)
extra_layers = keras.layers.Flatten()(extra_layers)
extra_layers = keras.layers.Dense(1024, activation='relu')(extra_layers)
extra_layers = keras.layers.Dropout(0.2)(extra_layers)

predictions = keras.layers.Dense(num_classes, activation='softmax')(extra_layers)

# create the new model
dropout_model = keras.models.Model(inputs=vgg_conv.input, outputs=predictions)

# compile the model
dropout_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
dropout_model.fit(train_images, epochs=num_epochs)

# print(train_images)

Epoch 1/10
1/1 [==============================] - 21s 21s/step - loss: 48.9387 - accuracy: 0.0123
Epoch 2/10
1/1 [==============================] - 18s 18s/step - loss: 45.7104 - accuracy: 0.0864
Epoch 3/10
1/1 [==============================] - 18s 18s/step - loss: 49.2377 - accuracy: 0.1605
Epoch 4/10
1/1 [==============================] - 18s 18s/step - loss: 48.5817 - accuracy: 0.2222
Epoch 5/10
1/1 [==============================] - 18s 18s/step - loss: 42.3200 - accuracy: 0.3086
Epoch 6/10
1/1 [==============================] - 18s 18s/step - loss: 36.6406 - accuracy: 0.3704
Epoch 7/10
1/1 [==============================] - 20s 20s/step - loss: 29.1919 - accuracy: 0.4198
Epoch 8/10
1/1 [==============================] - 18s 18s/step - loss: 24.6044 - accuracy: 0.5185
Epoch 9/10
1/1 [==============================] - 18s 18s/step - loss: 17.6654 - accuracy: 0.5679
Epoch 10/10
1/1 [==============================] - 18s 18s/step - loss: 12.7549 - accuracy: 0.6790


In [88]:
dropout_model.fit(train_images, epochs=num_epochs)

Epoch 1/10
1/1 [==============================] - 17s 17s/step - loss: 8.5352 - accuracy: 0.7284
Epoch 2/10
1/1 [==============================] - 17s 17s/step - loss: 5.5952 - accuracy: 0.8395
Epoch 3/10
1/1 [==============================] - 17s 17s/step - loss: 4.2947 - accuracy: 0.8519
Epoch 4/10
1/1 [==============================] - 17s 17s/step - loss: 3.5760 - accuracy: 0.8765
Epoch 5/10
1/1 [==============================] - 17s 17s/step - loss: 4.7760 - accuracy: 0.7901
Epoch 6/10
1/1 [==============================] - 17s 17s/step - loss: 4.8036 - accuracy: 0.8025
Epoch 7/10
1/1 [==============================] - 18s 18s/step - loss: 2.0577 - accuracy: 0.9136
Epoch 8/10
1/1 [==============================] - 17s 17s/step - loss: 1.2691 - accuracy: 0.9136
Epoch 9/10
1/1 [==============================] - 17s 17s/step - loss: 0.9412 - accuracy: 0.9012
Epoch 10/10
1/1 [==============================] - 17s 17s/step - loss: 0.7351 - accuracy: 0.9259


# Using data augmentation

## Resize & Rescale Images

In [89]:
resize_and_rescale = tf.keras.Sequential([
    keras.layers.Rescaling(1./255)
])

train_images = train_images.map(lambda x, y: (resize_and_rescale(x), y))
test_images = train_images.map(lambda x, y: (resize_and_rescale(x), y))

## Data Augmentation

In [90]:
data_flip = tf.keras.Sequential([
  keras.layers.RandomFlip("horizontal_and_vertical")
])

data_rotation = tf.keras.Sequential([
  keras.layers.RandomRotation(0.2)
])

flipped_images = train_images.map(lambda x, y: (data_flip(x), y))
rotated_images = train_images.map(lambda x, y: (data_rotation(x), y))

train_images = train_images.concatenate(flipped_images)
train_images = train_images.concatenate(rotated_images)

In [91]:
# test normal model
new_model.fit(train_images, epochs=num_epochs)

Epoch 1/10
3/3 [==============================] - 55s 13s/step - loss: 5.3542 - accuracy: 0.0658
Epoch 2/10
3/3 [==============================] - 51s 13s/step - loss: 4.9281 - accuracy: 0.0988
Epoch 3/10
3/3 [==============================] - 52s 14s/step - loss: 4.5048 - accuracy: 0.1687
Epoch 4/10
3/3 [==============================] - 50s 13s/step - loss: 4.1060 - accuracy: 0.2428
Epoch 5/10
3/3 [==============================] - 51s 13s/step - loss: 3.7688 - accuracy: 0.2963
Epoch 6/10
3/3 [==============================] - 51s 14s/step - loss: 3.4213 - accuracy: 0.3909
Epoch 7/10
3/3 [==============================] - 52s 13s/step - loss: 3.1864 - accuracy: 0.4156
Epoch 8/10
3/3 [==============================] - 58s 15s/step - loss: 2.9437 - accuracy: 0.4774
Epoch 9/10
3/3 [==============================] - 53s 14s/step - loss: 2.7402 - accuracy: 0.5432
Epoch 10/10
3/3 [==============================] - 55s 14s/step - loss: 2.4581 - accuracy: 0.6173


In [92]:
# test dropout model
dropout_model.fit(train_images, epochs=num_epochs)

Epoch 1/10
3/3 [==============================] - 52s 14s/step - loss: 4.1986 - accuracy: 0.0823
Epoch 2/10
3/3 [==============================] - 58s 14s/step - loss: 3.9309 - accuracy: 0.1523
Epoch 3/10
3/3 [==============================] - 49s 13s/step - loss: 3.8945 - accuracy: 0.1934
Epoch 4/10
3/3 [==============================] - 46s 13s/step - loss: 3.7293 - accuracy: 0.2016
Epoch 5/10
3/3 [==============================] - 49s 14s/step - loss: 3.5191 - accuracy: 0.3292
Epoch 6/10
3/3 [==============================] - 53s 15s/step - loss: 3.2785 - accuracy: 0.3374
Epoch 7/10
3/3 [==============================] - 53s 14s/step - loss: 3.1235 - accuracy: 0.3621
Epoch 8/10
3/3 [==============================] - 53s 14s/step - loss: 2.8366 - accuracy: 0.4486
Epoch 9/10
3/3 [==============================] - 55s 14s/step - loss: 2.8375 - accuracy: 0.4074
Epoch 10/10
3/3 [==============================] - 54s 14s/step - loss: 2.6036 - accuracy: 0.4527
